# Notes:

Metrics to add:
- column for type of workout so we can predict in different workouts


Time sequence options:
- pad at the end of each sequence and build an LSTM RNN with a layer that ignores padded portions of the sequence and a bidirectional layer to retain context from each phase
    1) separate pitch into phases (wind up, sit down power, and throw maybe? or possibly: wind-up, stride, arm cocking, acceleration, release)
        - ENSURE phases are always started the same, this takes away the noise of different starts and is ESSENTIAL or just do tree based instead of lstm
        - Add phase ID (e.g., wind_up=0, acceleration=1, release=2) as a feature to the model.
    2) use time column to split into different sequences (per trial)
        - consider normalizing the time sequences to 0,1. This only makes sense if it's not already normalized, check first and check distance in time steps to ensure no noise
    3) pad at the end of each phase so that it each sequence matches the max sequence
        - Pad each phase separately to its own max length (e.g., wind-up padded to 20 steps, acceleration to 50 steps).
        - Use a hierarchical model (e.g., separate LSTM per phase) to avoid mixing padded values across phases.
    4) create masked neural network that is bidirectional for edge cases and masking so we exclude the padded steps
        For causal tasks, use unidirectional LSTMs.
        For non-causal tasks (e.g., post-pitch classification), use bidirectional layers.
            from tensorflow.keras.layers import Input, Masking, Bidirectional, LSTM, Dense

            # Phase-specific input (e.g., wind-up)
            inputs = Input(shape=(max_phase_length, num_features + 2))  # +2 for phase ID and normalized time
            masked = Masking(mask_value=-1.0)(inputs)  # Assume padded time = -1
            lstm_out = Bidirectional(LSTM(64))(masked)  
            outputs = Dense(1, activation='sigmoid')(lstm_out)
        Add attention mechanisms to explicitly highlight critical moments (e.g., release):
            python
            from tensorflow.keras.layers import Attention, Concatenate
            # Add attention to focus on key timesteps (e.g., release)
            context_vector = Attention()([lstm_out, lstm_out])
            outputs = Concatenate()([lstm_out, context_vector])


Other options to test
- Dynamic Time Warping (DTW): Align sequences non-linearly to a reference length.
    Use Case: Best for comparing shapes of biomechanical curves (e.g., elbow angle vs. time).
    Use DTW as a preprocessing step for similarity-based tasks (e.g., clustering pitches by motion pattern).

- Resampling: Interpolate sequences to a fixed length (e.g., 100 steps) using biomechanical curves’ shape.
    Use Case: Suitable when biomechanical patterns are time-invariant (e.g., joint angles during release).
    Use cubic spline interpolation instead of linear to preserve curve shape.


Key Tests to Validate Approach

    Phase Alignment Check:

        Plot the distribution of phase durations (e.g., wind-up: 0.2–0.3s, release: 0.1–0.15s).

        Ensure padding doesn’t exceed 20% of the phase’s natural duration.
Key Tests to Validate Approach

    Phase Alignment Check:

        Plot the distribution of phase durations (e.g., wind-up: 0.2–0.3s, release: 0.1–0.15s).

        Ensure padding doesn’t exceed 20% of the phase’s natural duration.

    Ablation Study:

        Compare performance of:
        a) Phase-padded LSTM
        b) Resampled LSTM
        c) DTW-aligned model

    Attention Visualization:

        Use libraries like tf-explain to confirm attention focuses on biomechanically critical moments (e.g., release).

    Noise Injection Test:

        Add Gaussian noise to padded regions and verify model robustness (accuracy shouldn’t degrade)
    Ablation Study:

        Compare performance of:
        a) Phase-padded LSTM
        b) Resampled LSTM
        c) DTW-aligned model

    Attention Visualization:

        Use libraries like tf-explain to confirm attention focuses on biomechanically critical moments (e.g., release).

    Noise Injection Test:

        Add Gaussian noise to padded regions and verify model robustness (accuracy shouldn’t degrade)

# Load in the datasets: summarized, emg left joined, bio interpolated

In [2]:
import pandas as pd

# Load EMG pitch data for validation not for use really
emg_pitch_data = pd.read_parquet('../data/processed/emg_pitch_data_processed.parquet')
print("\nEMG Pitch Data Columns and Unique Values:")
for col in emg_pitch_data.columns:
    print(f"\n{col}:")
    print(f"Unique values: {emg_pitch_data[col].unique()}")

# Load summarized biomechanical profile
# summarized_bio = pd.read_parquet('../data/processed/ml_datasets/summarized/final_summarized_biomechanical_profile.parquet')
# print("\nSummarized Biomechanical Profile Columns and Unique Values:") 
# for col in summarized_bio.columns:
#     print(f"\n{col}:") 
#     print(f"Unique values: {summarized_bio[col].unique()}")


# Granular Datasets:
# Resample/interpolate the biomechanics dataset to join evenly onto the EMG data: No changes
# Bio dataset with EMG filtered out dataset: is_interpolated filter: filter for non interpolated data if you want to take away emg data for a bio dataset without interpolated added columns (will filter out EMG so they are on the bio frequency)
# EMG dataset with phases added on: create a interpolated column list so we can differ that from the non and create a EMG dataset with pitch phases added on (creating the simplistic EMG dataset with phases added on, no fake data involved and straight to the muscles dataset)

# Load granular dataset with interpolated bio data
granular_data = pd.read_parquet('../data/processed/ml_datasets/final_inner_join_emg_biomech_data.parquet')
print("\nGranular Dataset Columns and Unique Values:")
for col in granular_data.columns:
    print(f"\n{col}:")
    print(f"Unique values: {granular_data[col].unique()}")








EMG Pitch Data Columns and Unique Values:

EMG 1 (mV) - FDS (81770):
Unique values: [-0.0391089 -0.0345769 -0.0142672 ...  0.0040284  0.1359579  0.0691539]

ACC X (G) - FDS (81770):
Unique values: [-0.9207153 -0.9168091 -0.8994141 ...  0.4370117 -1.8967285 -1.4609985]

ACC Y (G) - FDS (81770):
Unique values: [0.406311  0.4068604 0.4025269 ... 0.1681519 0.414917  0.5172119]

ACC Z (G) - FDS (81770):
Unique values: [-0.3459473 -0.3414917 -0.3353271 ...  0.190918   0.2297974  0.1813354]

GYRO X (deg/s) - FDS (81770):
Unique values: [ 33.5343513  32.5190849  32.        ... -92.5190811  57.7557259
  44.0534363]

GYRO Y (deg/s) - FDS (81770):
Unique values: [  -6.2442746   -7.8091602   -9.2748089 ... -236.870224  -212.427475
 -107.2900772]

GYRO Z (deg/s) - FDS (81770):
Unique values: [-17.1832066 -17.4503822 -17.5496178 ...  69.3587799  64.8473282
  95.5267181]

EMG 1 (mV) - FCU (81728):
Unique values: [-0.008896   0.0159457  0.0287022 ...  0.336202   0.0241703 -0.207126 ]

ACC X (G) - FCU

EDA
Check all the basics:
- p value for each metric for suspicious stuff
- outliers
- standardization
- normalization
- dispersion
- disparity
- nulls and which to take out or to impute
- anything else that fixes this dataset 

In [3]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.decomposition import PCA

def detailed_eda(df, dataset_name="Dataset", display_plots=False):
    """
    Perform a detailed Exploratory Data Analysis (EDA) on the given DataFrame.
    
    Additional features added:
    - Duplicate row checks.
    - Consistency checks for datetime columns.
    - Visual Exploratory Analysis (histograms, density plots, Q-Q plots, boxplots).
    - Multicollinearity analysis using Variance Inflation Factor (VIF).
    - Basic time series checks: Stationarity (ADF test) and ACF/PACF plots.
    - PCA analysis for dimensionality reduction.
    - Domain-specific range checks for EMG, acceleration, and rotational columns.
    
    Parameters:
    - df: pandas DataFrame to analyze.
    - dataset_name: a string name for the dataset (used in print statements).
    - display_plots: if True, displays matplotlib plots (set to False for non-interactive environments).
    """
    print(f"\n\n----- Detailed EDA for {dataset_name} -----")
    
    # ---------------------------
    # Basic Data Overview
    # ---------------------------
    print("Shape:", df.shape)
    
    # Duplicate row check
    dup_count = df.duplicated().sum()
    print(f"\nDuplicate Rows: {dup_count} duplicate rows found.")
    
    print("\nDataFrame Info:")
    df.info()
    
    print("\nDescriptive Statistics:")
    print(df.describe(include='all').T)
    
    # ---------------------------
    # Nulls Check
    # ---------------------------
    print("\nMissing Values:")
    missing = df.isnull().sum()
    missing = missing[missing > 0]
    if missing.empty:
        print("No missing values found.")
    else:
        print(missing)
    
    # ---------------------------
    # Identify Numeric and Datetime Columns
    # ---------------------------
    numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
    # Filter out timedelta columns if any
    numeric_cols = [col for col in numeric_cols if not pd.api.types.is_timedelta64_dtype(df[col])]
    
    datetime_cols = df.select_dtypes(include=['datetime', 'datetimetz']).columns.tolist()
    
    # ---------------------------
    # Check for Datetime Consistency
    # ---------------------------
    if datetime_cols:
        print("\nDatetime Consistency Checks:")
        for col in datetime_cols:
            # Check if the datetime column is sorted
            if not df[col].is_monotonic_increasing:
                print(f"Warning: The datetime column '{col}' is not in increasing order.")
            else:
                print(f"The datetime column '{col}' is properly ordered.")
    
    # ---------------------------
    # Normality Test (Shapiro-Wilk) for Numeric Columns
    # ---------------------------
    print("\nNormality Test (Shapiro-Wilk) for numeric columns (p-values):")
    for col in numeric_cols:
        try:
            data = df[col].dropna()
            if len(data) > 5000:
                data = data.sample(5000, random_state=42)
            stat, p_value = stats.shapiro(data)
            print(f"{col}: p-value = {p_value:.4f}")
        except Exception as e:
            print(f"{col}: Error in normality test: {e}")
    
    # ---------------------------
    # Outlier Detection using IQR Method
    # ---------------------------
    print("\nOutlier Detection using IQR method:")
    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outlier_count = df[(df[col] < lower_bound) | (df[col] > upper_bound)].shape[0]
        print(f"{col}: {outlier_count} outliers detected.")
    
    # ---------------------------
    # Demonstrate Standardization and Normalization
    # ---------------------------
    print("\nStandardization (first 5 rows, Z-scores):")
    standardized = df[numeric_cols].apply(lambda x: (x - x.mean()) / x.std())
    print(standardized.head())
    
    print("\nNormalization (first 5 rows, scaled between 0 and 1):")
    normalized = df[numeric_cols].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
    print(normalized.head())
    
    # ---------------------------
    # Dispersion Metrics for Numeric Columns
    # ---------------------------
    print("\nDispersion and Disparity Metrics for numeric columns:")
    for col in numeric_cols:
        mean_val = df[col].mean()
        std_val = df[col].std()
        var_val = df[col].var()
        min_val = df[col].min()
        max_val = df[col].max()
        range_val = max_val - min_val
        print(f"{col}: mean = {mean_val:.4f}, std = {std_val:.4f}, variance = {var_val:.4f}, range = {range_val:.4f}")
    
    # ---------------------------
    # Correlation Matrix
    # ---------------------------
    print("\nCorrelation Matrix (numeric columns):")
    corr_matrix = df[numeric_cols].corr()
    print(corr_matrix)
    
    # ---------------------------
    # Visual Exploratory Analysis
    # ---------------------------
    print("\nGenerating visual exploratory plots (if display_plots=True)...")
    if display_plots:
        for col in numeric_cols:
            plt.figure(figsize=(12, 4))
            
            # Histogram and density plot
            plt.subplot(1, 3, 1)
            sns.histplot(df[col].dropna(), kde=True)
            plt.title(f'Histogram & KDE: {col}')
            
            # Q-Q plot
            plt.subplot(1, 3, 2)
            stats.probplot(df[col].dropna(), dist="norm", plot=plt)
            plt.title(f'Q-Q Plot: {col}')
            
            # Boxplot
            plt.subplot(1, 3, 3)
            sns.boxplot(x=df[col].dropna())
            plt.title(f'Boxplot: {col}')
            
            plt.tight_layout()
            plt.show()
    
    # ---------------------------
    # Multicollinearity Check using VIF
    # ---------------------------
    if len(numeric_cols) > 1:
        print("\nMulticollinearity Check using VIF:")
        # Prepare data for VIF calculation (drop missing values)
        df_numeric = df[numeric_cols].dropna()
        vif_data = pd.DataFrame()
        vif_data["Feature"] = df_numeric.columns
        vif_data["VIF"] = [variance_inflation_factor(df_numeric.values, i) 
                           for i in range(len(df_numeric.columns))]
        print(vif_data)
    
    # ---------------------------
    # Time Series Specific Checks (if applicable)
    # ---------------------------
    if datetime_cols:
        print("\nTime Series Analysis:")
        # For each datetime column, try to find an associated numeric column for time series checks.
        for dt_col in datetime_cols:
            # Find numeric columns that might be associated (heuristic: columns that are not IDs)
            candidate_cols = [col for col in numeric_cols if 'id' not in col.lower()]
            for num_col in candidate_cols:
                # Sort by datetime column
                ts_data = df[[dt_col, num_col]].dropna().sort_values(by=dt_col)
                if ts_data.shape[0] > 30:  # ensure enough data points
                    # Stationarity test: ADF test
                    try:
                        adf_result = adfuller(ts_data[num_col])
                        print(f"ADF Test for {num_col} (sorted by {dt_col}): p-value = {adf_result[1]:.4f}")
                    except Exception as e:
                        print(f"Error performing ADF test on {num_col}: {e}")
                    
                    # If display_plots is True, plot ACF and PACF
                    if display_plots:
                        fig, axes = plt.subplots(1, 2, figsize=(12, 4))
                        plot_acf(ts_data[num_col], ax=axes[0], lags=20)
                        axes[0].set_title(f'ACF of {num_col}')
                        plot_pacf(ts_data[num_col], ax=axes[1], lags=20)
                        axes[1].set_title(f'PACF of {num_col}')
                        plt.tight_layout()
                        plt.show()
                    # Only process one candidate per datetime column
                    break

    # ---------------------------
    # PCA Analysis for Numeric Data
    # ---------------------------
    if len(numeric_cols) > 1:
        print("\nPCA Analysis on numeric columns:")
        # Drop NA values for PCA and standardize data
        pca_data = df[numeric_cols].dropna()
        standardized_data = (pca_data - pca_data.mean()) / pca_data.std()
        try:
            pca = PCA()
            pca.fit(standardized_data)
            explained_variance = pca.explained_variance_ratio_
            for idx, var in enumerate(explained_variance):
                print(f"PC{idx+1}: {var*100:.2f}% of variance explained")
        except Exception as e:
            print("Error performing PCA:", e)
    
    # ---------------------------
    # Domain-Specific Range Checks
    # ---------------------------
    print("\nDomain-Specific Range Checks:")
    domain_keywords = ['emg', 'accel', 'rotat']
    for col in numeric_cols:
        if any(keyword in col.lower() for keyword in domain_keywords):
            col_min = df[col].min()
            col_max = df[col].max()
            print(f"{col} (domain-specific): min = {col_min}, max = {col_max}")
            # Here you can add further domain-specific validations if needed.
    
    print("\n----- End of EDA for", dataset_name, "-----\n")


# ------------------ New EDA Calls ------------------
# Run the detailed EDA on each dataset
# detailed_eda(emg_pitch_data, dataset_name="EMG Pitch Data")
# detailed_eda(summarized_bio, dataset_name="Summarized Biomechanical Profile")
detailed_eda(granular_data, dataset_name="Granular Dataset")




----- Detailed EDA for Granular Dataset -----
Shape: (134720, 115)

Duplicate Rows: 0 duplicate rows found.

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134720 entries, 0 to 134719
Columns: 115 entries, EMG 1 (mV) - FDS (81770) to time_difference
dtypes: bool(1), datetime64[ns](4), float64(69), int64(19), object(18), timedelta64[ns](4)
memory usage: 117.3+ MB

Descriptive Statistics:
                                 count unique         top    freq  \
EMG 1 (mV) - FDS (81770)      134720.0    NaN         NaN     NaN   
ACC X (G) - FDS (81770)       134720.0    NaN         NaN     NaN   
ACC Y (G) - FDS (81770)       134720.0    NaN         NaN     NaN   
ACC Z (G) - FDS (81770)       134720.0    NaN         NaN     NaN   
GYRO X (deg/s) - FDS (81770)  134720.0    NaN         NaN     NaN   
...                                ...    ...         ...     ...   
time_step_biomech               134720    NaN         NaN     NaN   
date_biomech                    13472

c:\Users\GeoffreyHadfield\Anaconda3\envs\data_science_fatigue_analysis\lib\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


ACC X (G) - FDS (81770)_spike_flag: p-value = 0.0000
ACC X (G) - FCU (81728)_spike_flag: p-value = 0.0000
ACC Y (G) - FDS (81770)_spike_flag: p-value = 0.0000
ACC Y (G) - FCU (81728)_spike_flag: p-value = 0.0000
ACC Z (G) - FDS (81770)_spike_flag: p-value = 0.0000
ACC Z (G) - FCU (81728)_spike_flag: p-value = 0.0000
GYRO X (deg/s) - FDS (81770)_spike_flag: p-value = 0.0000
GYRO X (deg/s) - FCU (81728)_spike_flag: p-value = 0.0000
GYRO Y (deg/s) - FDS (81770)_spike_flag: p-value = 0.0000
GYRO Y (deg/s) - FCU (81728)_spike_flag: p-value = 0.0000
GYRO Z (deg/s) - FDS (81770)_spike_flag: p-value = 0.0000
GYRO Z (deg/s) - FCU (81728)_spike_flag: p-value = 0.0000
EMG 1 (mV) - FDS (81770)_spike_flag: p-value = 0.0000
EMG_high_flag: p-value = 0.0000
EMG_low_flag: p-value = 0.0000
EMG_extreme_flag: p-value = 0.0000
EMG_extreme_flag_dynamic: p-value = 1.0000
ThrowingMotion: p-value = 0.0000
session_biomech: p-value = 1.0000
trial_biomech: p-value = 0.0000
ongoing_timestamp_biomech: p-value = 1.0

c:\Users\GeoffreyHadfield\Anaconda3\envs\data_science_fatigue_analysis\lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
c:\Users\GeoffreyHadfield\Anaconda3\envs\data_science_fatigue_analysis\lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


                                        Feature       VIF
0                      EMG 1 (mV) - FDS (81770)  1.000002
1                       ACC X (G) - FDS (81770)  1.000055
2                       ACC Y (G) - FDS (81770)  1.000010
3                       ACC Z (G) - FDS (81770)  1.000000
4                  GYRO X (deg/s) - FDS (81770)  1.000110
..                                          ...       ...
83            phase_weighted_cumulative_biomech  1.002456
84                    cumulative_valgus_biomech  1.000630
85                       critical_phase_biomech  1.022390
86  cumulative_valgus_phase_armcock_acc_biomech  1.101465
87                   peak_torque_marker_biomech  1.117854

[88 rows x 2 columns]

Time Series Analysis:
ADF Test for EMG 1 (mV) - FDS (81770) (sorted by Date/Time_parsed): p-value = 0.0000
ADF Test for EMG 1 (mV) - FDS (81770) (sorted by Timestamp_parsed): p-value = 0.0000
ADF Test for EMG 1 (mV) - FDS (81770) (sorted by datetime): p-value = 0.0000
ADF Test fo

# Feature Engineering

metrics to add:
* Set up phases based on the pitcher movement so we can set them into different phases
* add the phases to the EMG dataset so we can better predict on that and create use the merge_asof dataset to see if it's worth predicting with for the time series motion of the joints (the merge_asof would need to be a small difference)
* add in pulse data if we don't have arm slot/torque/speed in the database
* add in vulgas range and possibly the ongoing detriment from that
* finish up the sql statement (ensure it's completely validated) for the base table
* separate the dataset into time series classification, regression and tree based classification datasets
* check the feaures importances and such
* use permutation importance and use github(dot)com/eli5-org/eli5

Setting up feature dictionary to speak over each feature to talk about actual important metrics outside of those feature selected
Feature Selection (RFE, SHAP importance, correlation (to take out multi-collinearity metrics or combine them))


In [ ]:
# ### 1. Composite Exhaustion Index

# - **What It Is:**  
#   This metric combines three EMG‐based fatigue indicators (activation decay, frequency shift, and force–activation efficiency) into a single continuous score. In your sample output, you see values such as –0.314, –1.866, etc.

# - **Use in Machine Learning:**  
#   - **Regression:** You could use this index as a target variable to forecast the level of muscle fatigue over time (e.g., predicting how fatigue will progress during a session).  
#   - **Classification:** By setting a threshold (as in your muscle injury flag), you can create binary labels (injury risk vs. no risk) for classification tasks.  
#   - **Feature:** Even if not the target, the exhaustion index can serve as an informative feature that explains variation in performance or injury occurrence.

# - **Insights:**  
#   A lower (or more negative) composite index might indicate lower relative activation (or greater fatigue) once the inversion is applied. Understanding its distribution over time could help in dynamic scheduling or real-time risk monitoring.

# ---

# ### 2. Joint Angle Extremity Index

# - **What It Is:**  
#   This index represents the percentage of time a joint (here, the elbow) spends at its extreme ranges of motion. In your output, the composite index is 20.0, meaning that, on average, the joint is near its extreme values 20% of the time.

# - **Use in Machine Learning:**  
#   - **Regression:** As a continuous feature, it can help predict outcomes such as performance degradation or the likelihood of an injury.  
#   - **Classification:** A high extremity index could be used to flag potential injury risk. For instance, if a subject frequently reaches extreme ROM, they might be classified as “at risk.”

# - **Insights:**  
#   Extremity time might correlate with fatigue or overuse injuries. It also can help in training programs by indicating whether an athlete is pushing their joint limits too often.

# ---

# ### 3. ROM Progression Metric

# - **What It Is:**  
#   This metric tracks the progression (or change) in the maximum joint angle over time using a rolling window. Small incremental changes (e.g., a progression of 0.000627) indicate how the range of motion evolves during a session.

# - **Use in Machine Learning:**  
#   - **Regression:** It can be used as a predictor for changes in performance. For example, you might forecast the point at which an athlete’s joint movement begins to degrade.  
#   - **Classification:** Sudden changes in ROM progression might indicate abnormal movement patterns or an impending injury, allowing for early warnings.

# - **Insights:**  
#   Monitoring progression can reveal both adaptation (e.g., improvement) and fatigue (e.g., reduced ROM) over time, making it valuable for performance tracking and injury prevention.

# ---

# ### 4. Muscle Overextension Data

# - **What It Is:**  
#   This data estimates the percentage of time a muscle is operating beyond its safe length (over 120% of its rest length in your dummy example). With values near 99.86% in your output, it may indicate that the calculation or threshold requires adjustment for your application.

# - **Use in Machine Learning:**  
#   - **Regression:** It can help predict muscle strain or the onset of injury based on overextension patterns.  
#   - **Classification:** Setting an appropriate threshold could allow you to classify segments of the time series as “safe” versus “risky” in terms of muscle loading.

# - **Insights:**  
#   Even if the current dummy values need calibration, this metric could serve as an important indicator of biomechanical stress, contributing to injury risk models.

# ---

# ### 5. Velocity at Extreme ROM

# - **What It Is:**  
#   This feature measures the angular velocity when the joint is at its extreme ranges. With a composite value of around 169.3 (mean velocity) and a maximum value that reaches into the thousands, it indicates how rapidly the joint is moving during these critical phases.

# - **Use in Machine Learning:**  
#   - **Regression:** Use this as a continuous feature to forecast the risk of injury based on rapid, high-velocity movements at extreme positions.  
#   - **Classification:** High velocities at extremes could be thresholds for injury risk, leading to a binary risk indicator.

# - **Insights:**  
#   Rapid movements at extreme angles are often linked to acute injury risk. Including this metric can enhance models aimed at real-time monitoring or early intervention.

# ---

# ### 6. Injury Indicators

# - **Elbow Injury Flag:**  
#   Created by flagging when the elbow angle exceeds a threshold (e.g., 120°), this indicator is useful as a binary label.  
#   - **Classification Use:** It directly provides a ground truth label (injury risk: 1 vs. safe: 0) that can be used to train supervised models.

# - **Muscle Injury Flag:**  
#   Based on whether the composite exhaustion index exceeds a threshold, this flag provides another injury risk indicator.  
#   - **Multi-Factor Analysis:** It can be combined with other physiological or kinematic data to improve the accuracy of injury prediction models.

# ---

# ### Additional Use Cases

# 1. **Anomaly Detection:**  
#    By modeling the expected behavior of these features (using autoencoders or clustering methods), you could flag unusual patterns in muscle fatigue or joint mechanics that might not be immediately evident from the raw data.

# 2. **Session Segmentation:**  
#    The evolution of these metrics over time can help segment sessions into phases (e.g., warm-up, peak performance, fatigue, recovery) which can be useful for personalized training recommendations.

# 3. **Real-Time Monitoring and Alerts:**  
#    In a wearable or sports analytics system, these metrics can feed into a dashboard that provides real-time feedback on fatigue and injury risk, triggering alerts if thresholds are exceeded.

# 4. **Predictive Maintenance for Athletes:**  
#    Using time series regression, you can forecast the need for rest or adjustments in training based on predicted fatigue levels, thereby preventing overtraining injuries.

# 5. **Comparative Analysis Across Sessions or Athletes:**  
#    You can aggregate these features to compare how different athletes or training sessions differ in terms of fatigue progression and injury risk, enabling data-driven coaching strategies.

# ---

# ### Conclusion

# Each of these new statistics brings a unique perspective to the data:

# - **Composite Exhaustion Index:** Offers an aggregated measure of fatigue.
# - **Joint Angle Extremity & ROM Progression Metrics:** Reveal movement patterns and changes over time.
# - **Muscle Overextension & Velocity at Extreme ROM:** Provide biomechanical stress indicators.
# - **Injury Flags:** Serve as labels or targets for predictive models.

# By combining these features, you can build robust regression or classification models that predict outcomes such as fatigue levels, injury risk, or performance decline. Moreover, these features can be integrated into real-time systems for monitoring and decision-making, as well as for post-session analytics to improve training regimens.



#---------------------------
# Forecasting Pitcher Fatigue Using Time Series Analysis: A Joint-by-Joint and Muscle-by-Muscle Approach

# The ability to predict how fatigue will accumulate in a pitcher's body over time represents one of the most promising frontiers in sports biomechanics and injury prevention. Based on the biomechanical metrics and fatigue indicators identified in the research, we can develop sophisticated time series forecasting models that predict fatigue accumulation at the level of individual joints and muscles, potentially revolutionizing pitcher management and injury prevention.

# ## Time Series Structures for Pitching Session Fatigue

# To effectively forecast fatigue, we must first properly structure our time series data to capture the complex temporal dynamics of fatigue development. This involves multiple time scales and resolutions that each provide unique insights.

# ### Multi-Resolution Temporal Framework

# The most effective approach would involve nested time series structures that capture fatigue at different temporal resolutions:

# 1. **Pitch-by-pitch sequence** - The finest granularity, capturing acute fatigue development during a single session
# 2. **Session-by-session sequence** - Medium granularity, tracking recovery and fatigue across consecutive pitching sessions
# 3. **Season-long sequence** - Coarse granularity, identifying long-term cumulative fatigue patterns

# Dr. Michael Sonne's work on Fatigue Units demonstrates how important the timing between pitches is for understanding fatigue accumulation. His research shows that the average time between pitches in a game (15 seconds) versus during warmups (3.5 seconds) creates dramatically different fatigue profiles, even with the same number of pitches[11]. This suggests that our time series model must incorporate not just the number of pitches, but also the temporal spacing between them.

# ### Feature Engineering for Comprehensive Fatigue Assessment

# Each data point in our time series would need to include a rich set of features that comprehensively capture the biomechanical state of the pitcher:

# 1. **EMG-derived fatigue indicators**:
#    - Activation decay rates for key muscles
#    - Frequency shift in muscle activation patterns
#    - Force-activation efficiency metrics

# 2. **Kinematic metrics**:
#    - Joint angle extremity indices
#    - ROM progression metrics
#    - Hip-to-shoulder separation percentages
#    - Knee flexion angles at ball release

# 3. **Kinetic and performance metrics**:
#    - Velocity at extreme ROM
#    - Pitch velocity
#    - Accuracy metrics
#    - Recovery time between pitches

# Research by Murray et al. found that fatigued pitchers showed significantly less maximum shoulder external rotation and knee flexion at ball release, with reduced shoulder and elbow proximal force[9]. Similarly, work by Escamilla et al. revealed that fatigued pitchers adopted a more upright trunk position[9]. These specific mechanical changes provide concrete features our time series model could track to identify fatigue patterns.

# ## Joint-Specific and Muscle-Specific Forecasting Models

# One of the most powerful aspects of this approach is the ability to develop fatigue forecasts that are specific to individual joints and muscles, rather than relying on general fatigue indicators.

# ### Joint-Specific Fatigue Forecasting

# For each key joint involved in pitching (shoulder, elbow, wrist, hip, knee, ankle), we can develop separate forecasting models that predict:

# 1. **Joint-specific mechanical fatigue** - How movement patterns at that joint will likely change in future sessions
# 2. **Joint loading patterns** - Predicted forces and torques at each joint
# 3. **Injury risk thresholds** - When joint mechanics may approach dangerous levels

# The Joint Angle Extremity Index offers an excellent foundation for this work, as it quantifies the percentage of time joints spend at extreme ranges of motion[1]. This metric could be forecasted for future pitching sessions, with increasing values potentially indicating greater injury risk. The data shows that in some samples, joints spend around 20% of their time at extreme positions[1], and predicting when this percentage might increase could trigger preventive interventions.

# ### Muscle-Specific Fatigue Forecasting

# Similarly, for key muscle groups (rotator cuff, flexor-pronator mass, core muscles), we can forecast:

# 1. **Force production capacity** - How much strength each muscle group will retain
# 2. **Activation pattern changes** - How recruitment patterns will shift with fatigue
# 3. **Recovery trajectories** - How quickly each muscle group will recover between sessions

# The Composite Exhaustion Index, which combines activation decay, frequency shift, and force-activation efficiency into a single metric, provides an excellent foundation for muscle fatigue forecasting[1]. Values in the sample data range from -0.314464 to -1.866018, indicating varying degrees of muscle fatigue[1]. By forecasting these values, we could predict which muscles will become critically fatigued in future sessions.

# ## Advanced Time Series Modeling Approaches

# To effectively forecast these complex, multivariate fatigue patterns, we need sophisticated time series modeling approaches. 

# ### Long Short-Term Memory (LSTM) Networks

# LSTMs are particularly well-suited for this application because they can:

# 1. Capture long-range dependencies in the data
# 2. Model complex, non-linear relationships
# 3. Handle multivariate inputs and outputs
# 4. Learn patterns of fatigue accumulation and recovery

# Recent research using LSTM networks with IMU-generated multivariate time series data demonstrated "high predictive accuracy for fatigue, showing significant correlations between predicted fatigue levels and observed declines in performance"[13]. This approach enabled "individualized training adjustments that were in sync with athletes' physiological thresholds"[13].

# ### Ensemble Methods for Robust Forecasting

# Combining multiple forecasting models can provide more robust predictions:

# 1. **Random Forests** - For identifying critical thresholds and non-linear relationships
# 2. **Gradient Boosting Machines** - For capturing complex interactions between features
# 3. **LSTM Networks** - For modeling temporal dependencies

# The comparative analysis of these approaches in the IMU fatigue study demonstrated that ensemble methods could effectively minimize systematic prediction errors through bias correction mechanisms[13].

# ## Personalization Through Pitcher Grouping and Clustering

# Not all pitchers fatigue in the same way or at the same rate. Some may show fatigue first in their shoulder, while others might demonstrate early signs in their elbow mechanics or lower body.

# ### Clustering Pitchers by Fatigue Response Patterns

# We can use unsupervised learning techniques to identify distinct "fatigue profiles" among pitchers:

# 1. **Mechanical Degradation Clusters** - Groups based on which mechanical elements decline first
# 2. **Recovery Rate Clusters** - Groups based on how quickly pitchers recover from fatigue
# 3. **Injury Risk Clusters** - Groups based on mechanical patterns associated with injury risk

# By identifying which cluster a pitcher belongs to, we can select the most appropriate forecasting model for their specific fatigue pattern.

# ### Adaptive Models for Individual Pitchers

# As more data is collected for an individual pitcher, the forecasting models can be tuned to their unique fatigue patterns:

# 1. **Personalized Baseline Parameters** - Adjusted based on individual mechanics
# 2. **Customized Warning Thresholds** - Set according to personal injury risk factors
# 3. **Pitcher-Specific Recovery Models** - Calibrated to individual recovery rates

# This personalization aspect is crucial, as research shows significant individual variation in how pitchers respond to fatigue. A study of adolescent pitchers found that while they generally became more fatigued (0.3 ± 0.6 to 3.5 ± 2.1 on a visual analog scale) and experienced more pain (0.1 ± 0.4 to 1.6 ± 2.2) as pitch count increased, the specific mechanical changes varied considerably between pitchers[14].

# ## Implementation Strategy for Pitcher Fatigue Forecasting

# ### Data Collection Requirements

# To implement this forecasting approach, we would need:

# 1. **High-frequency biomechanical data** - Collected via motion capture or IMUs
# 2. **EMG measurements** - For key muscle groups involved in pitching
# 3. **Performance metrics** - Including velocity, accuracy, and subjective fatigue ratings
# 4. **Recovery indicators** - Such as HRV, sleep quality, and soreness levels

# ### Model Training and Validation Process

# The forecasting system would be developed through:

# 1. **Historical data analysis** - Identifying patterns in past pitching sessions
# 2. **Initial model training** - Using data from multiple pitchers
# 3. **Cross-validation** - Testing prediction accuracy on held-out sessions
# 4. **Model refinement** - Tuning parameters based on validation results
# 5. **Individual calibration** - Adjusting models for specific pitchers

# ### Real-Time Feedback and Intervention System

# Once deployed, the system would:

# 1. **Monitor fatigue in real-time** - Tracking key metrics during pitching sessions
# 2. **Forecast future fatigue states** - Predicting how fatigue will develop in upcoming sessions
# 3. **Generate adaptive recommendations** - Suggesting modifications to workload or mechanics
# 4. **Optimize recovery protocols** - Personalizing between-session recovery approaches

# ## Conclusion

# Forecasting pitcher fatigue at the joint and muscle level represents a significant advancement in sports science and injury prevention. By leveraging the rich biomechanical metrics identified in the research and applying sophisticated time series modeling approaches, we can predict how fatigue will accumulate and manifest in future pitching sessions.

# This approach offers several key benefits:

# 1. **Proactive injury prevention** - Identifying fatigue-related risk before symptoms appear
# 2. **Optimized training programs** - Customizing workloads based on predicted fatigue states
# 3. **Personalized recovery strategies** - Tailoring interventions to specific fatigue patterns
# 4. **Performance enhancement** - Maintaining mechanical efficiency despite accumulated workload

# As Dr. Sonne's research on fatigue modeling demonstrates, "muscles need time to recover from fatigue"[11], and by properly modeling these fatigue and recovery dynamics across multiple time scales, we can develop a powerful forecasting system that revolutionizes how pitchers are managed throughout a season, potentially reducing injuries while maximizing performance.

# Citations:
# [1] https://ppl-ai-file-upload.s3.amazonaws.com/web/direct-files/53658542/aaba7b9e-6dbd-4943-a123-41f2f4058820/paste.txt
# [2] https://ppl-ai-file-upload.s3.amazonaws.com/web/direct-files/53658542/3ebccbe4-975f-4c29-a28b-0c4fabadd387/paste-2.txt
# [3] https://fantasy.fangraphs.com/an-introduction-to-fatigue-units-a-new-method-for-evaluating-workloads/
# [4] https://pubmed.ncbi.nlm.nih.gov/16973902/
# [5] https://www.bearcognition.com/post/predicting-athlete-fatigue-with-regression-models
# [6] https://www.tandfonline.com/doi/pdf/10.1080/02640414.2016.1150600
# [7] http://www.mikesonne.ca/baseball/giving-a-big-fu-to-workload-metrics-in-pitchers-part-1-fatigue-modelling/
# [8] https://digital.wpi.edu/show/n870zv415
# [9] https://pmc.ncbi.nlm.nih.gov/articles/PMC3445126/
# [10] http://www.mikesonne.ca/baseball/using-gameday-data-for-fatigue-modelling/
# [11] https://www.drivelinebaseball.com/2019/09/motus-advances-workload-models-with-fatigue-units/
# [12] https://pmc.ncbi.nlm.nih.gov/articles/PMC4555605/
# [13] https://pubmed.ncbi.nlm.nih.gov/38202992/
# [14] https://www.sportssurgerychicago.com/2016/05/24/impact-fatigue-baseball-pitching-mechanics-adolescent-male-pitchers/
# [15] https://blog.armcare.com/unraveling-the-science-of-throwing-arm-fatigue-in-pitchers/
# [16] https://pmc.ncbi.nlm.nih.gov/articles/PMC8711585/
# [17] https://pmc.ncbi.nlm.nih.gov/articles/PMC6673423/
# [18] https://www.drivelinebaseball.com/pitching-research-biomechanics/
# [19] https://pubmed.ncbi.nlm.nih.gov/15611006/
# [20] https://baseballwithr.wordpress.com/2022/10/20/studying-pitcher-fatigue-using-a-multinomial-regression-model/
# [21] https://stats100blog.wordpress.com/2023/06/09/impact-of-pitcher-fatigue-in-the-mlb/
# [22] https://www.researchgate.net/publication/6817281_Pitching_Biomechanics_as_a_Pitcher_Approaches_Muscular_Fatigue_During_a_Simulated_Baseball_Game
# [23] https://sabr.org/journal/article/artificial-intelligence-machine-learning-and-the-bright-future-of-baseball/
# [24] https://peerj.com/articles/7390/
# [25] https://journals.lww.com/nsca-scj/fulltext/2014/12000/Monitoring_and_Managing_Fatigue_in_Baseball.4.aspx?generateEpub=Article%7Cnsca-scj%3A2014%3A12000%3A00004%7C10.1519%2Fssc.0000000000000100%7C
# [26] https://digitalcommons.wku.edu/ijesab/vol16/iss3/18/
# [27] https://www.frontiersin.org/journals/psychology/articles/10.3389/fpsyg.2021.741805/full
# [28] https://journals.lww.com/nsca-jscr/fulltext/2024/08000/acute_upper_body_and_lower_body_neuromuscular.13.aspx
# [29] https://journals.lww.com/nsca-jscr/fulltext/9900/acute_upper_body_and_lower_body_neuromuscular.453.aspx
# [30] https://opensportssciencesjournal.com/VOLUME/13/PAGE/66/FULLTEXT/
# [31] https://commons.nmu.edu/cgi/viewcontent.cgi?article=2611&context=isbs
# [32] https://journals.sagepub.com/doi/abs/10.1177/0363546506293025
# [33] https://journals.sagepub.com/doi/pdf/10.1177/2325967114537032
# [34] https://journals.sagepub.com/doi/10.1177/0363546504266071

# ---
# Answer from Perplexity: pplx.ai/share


def activation_decay_index(emg_data, muscle_cols=['FDS', 'FCU', 'FCR'], window_size=60):
    """
    Calculate the normalized rate of EMG amplitude decline for each muscle.
    
    Parameters:
    - emg_data: DataFrame containing EMG data for specified muscles.
    - muscle_cols: List of column names for target muscles.
    - window_size: Size of rolling window (in samples or seconds, as applicable).
    
    Returns:
    - DataFrame with decay rates for each muscle and a composite score.
    """
    results = pd.DataFrame()
    
    for muscle in muscle_cols:
        # Calculate initial activation level from the first window
        initial_activation = emg_data[muscle].iloc[:window_size].mean()
        
        # Compute rolling mean activation using the given window size
        rolling_activation = emg_data[muscle].rolling(window=window_size).mean()
        
        # Normalize by the initial activation to express values as a percentage
        normalized_activation = rolling_activation / initial_activation
        
        decay_rates = []
        # Compute decay rate using linear regression on segments of the normalized activation
        for i in range(window_size, len(emg_data), window_size//2):
            segment = normalized_activation.iloc[max(0, i-window_size):i]
            # Ensure there are enough data points before performing regression
            if len(segment) >= window_size//2:
                x = np.arange(len(segment))
                slope, _, _, _, _ = stats.linregress(x, segment)
                decay_rates.append(slope)
            else:
                decay_rates.append(np.nan)
                
        results[f'{muscle}_decay_rate'] = decay_rates
    
    # Composite score: average decay rate across the muscles
    results['composite_decay_rate'] = results.mean(axis=1)
    
    return results

from scipy import signal  # Ensure signal module is imported

def frequency_shift_index(emg_data, muscle_cols=['FDS', 'FCU', 'FCR'], window_size=60, sampling_rate=1000):
    """
    Calculate the shift in median frequency of EMG signals as an indicator of fatigue.
    
    Parameters:
    - emg_data: DataFrame containing EMG data for specified muscles.
    - muscle_cols: List of column names for target muscles.
    - window_size: Size of the rolling window (in samples or seconds).
    - sampling_rate: Sampling rate of EMG signals in Hz.
    
    Returns:
    - DataFrame with median frequency values and normalized frequency ratios for each muscle.
    """
    results = pd.DataFrame()
    
    for muscle in muscle_cols:
        median_freqs = []
        for i in range(window_size, len(emg_data), window_size//2):
            segment = emg_data[muscle].iloc[max(0, i-window_size):i].values
            if len(segment) >= window_size//2:
                # Compute power spectrum using Welch's method
                f, Pxx = signal.welch(segment, fs=sampling_rate, nperseg=min(256, len(segment)))
                total_power = np.sum(Pxx)
                cumulative_power = np.cumsum(Pxx)
                median_freq_idx = np.where(cumulative_power >= total_power/2)[0][0]
                median_freq = f[median_freq_idx]
                median_freqs.append(median_freq)
            else:
                median_freqs.append(np.nan)
                
        results[f'{muscle}_median_freq'] = median_freqs
    
    # Normalize the median frequencies relative to the initial value to observe changes
    for muscle in muscle_cols:
        initial_freq = results[f'{muscle}_median_freq'].iloc[0]
        results[f'{muscle}_freq_ratio'] = results[f'{muscle}_median_freq'] / initial_freq
    
    # Composite frequency shift index: average across muscles
    results['composite_freq_shift'] = results[[f'{muscle}_freq_ratio' for muscle in muscle_cols]].mean(axis=1)
    
    return results


def force_activation_efficiency(emg_data, force_data, muscle_cols=['FDS', 'FCU', 'FCR'], window_size=60):
    """
    Calculate the efficiency ratio between force output and EMG activation.
    
    Parameters:
    - emg_data: DataFrame containing EMG data for specified muscles.
    - force_data: Series or DataFrame column containing force measurements.
    - muscle_cols: List of column names for target muscles.
    - window_size: Size of the rolling window (in samples or seconds).
    
    Returns:
    - DataFrame with efficiency ratios and normalized efficiency for each muscle over time.
    """
    results = pd.DataFrame()
    
    for muscle in muscle_cols:
        efficiency_values = []
        for i in range(window_size, len(emg_data), window_size//2):
            emg_segment = emg_data[muscle].iloc[max(0, i-window_size):i]
            force_segment = force_data.iloc[max(0, i-window_size):i]
            
            if len(emg_segment) >= window_size//2:
                mean_activation = emg_segment.mean()
                mean_force = force_segment.mean()
                # Compute efficiency as force per unit activation; add small constant to avoid division by zero
                efficiency = mean_force / (mean_activation + 1e-6)
                efficiency_values.append(efficiency)
            else:
                efficiency_values.append(np.nan)
                
        results[f'{muscle}_efficiency'] = efficiency_values
    
    # Normalize efficiency values relative to the initial value for each muscle
    for muscle in muscle_cols:
        initial_efficiency = results[f'{muscle}_efficiency'].iloc[0]
        results[f'{muscle}_efficiency_ratio'] = results[f'{muscle}_efficiency'] / initial_efficiency
    
    # Composite efficiency index: average of the normalized ratios across muscles
    results['composite_efficiency'] = results[[f'{muscle}_efficiency_ratio' for muscle in muscle_cols]].mean(axis=1)
    
    return results


def composite_exhaustion_index(decay_data, freq_data, efficiency_data, weights=[0.4, 0.3, 0.3]):
    """
    Create a comprehensive exhaustion index by combining multiple fatigue indicators.
    
    Parameters:
    - decay_data: DataFrame output from activation_decay_index.
    - freq_data: DataFrame output from frequency_shift_index.
    - efficiency_data: DataFrame output from force_activation_efficiency.
    - weights: List indicating the relative importance of [decay, frequency, efficiency].
    
    Returns:
    - Series containing the composite exhaustion scores.
    """
    # Invert frequency shift and efficiency ratio because lower values indicate fatigue in these measures
    inverted_freq_shift = 1 - freq_data['composite_freq_shift']
    inverted_efficiency = 1 - efficiency_data['composite_efficiency']
    
    # Normalize decay rate. Here, higher absolute decay indicates more fatigue so we normalize by its minimum value.
    norm_decay = decay_data['composite_decay_rate'] / decay_data['composite_decay_rate'].min()
    
    # Compute composite exhaustion index using the specified weights
    exhaustion_index = (weights[0] * norm_decay + 
                        weights[1] * inverted_freq_shift + 
                        weights[2] * inverted_efficiency)
    
    return exhaustion_index

def joint_angle_extremity_index(kinematic_data, joint_cols, percentile_threshold=90):
    """
    Calculate the percentage of time spent near extremes of range of motion (ROM) for each joint.
    
    Parameters:
    - kinematic_data: DataFrame containing joint angle data.
    - joint_cols: List of column names for joint angles.
    - percentile_threshold: Percentile used to define extreme ROM (e.g., 90 for top 10% extremes).
    
    Returns:
    - DataFrame with extremity indices for each joint and a composite index.
    """
    results = pd.DataFrame()
    
    for joint in joint_cols:
        # Define thresholds based on the given percentile
        max_threshold = np.percentile(kinematic_data[joint], percentile_threshold)
        min_threshold = np.percentile(kinematic_data[joint], 100 - percentile_threshold)
        
        # Calculate percentage of time joint angle is at or beyond the extreme thresholds
        extreme_max_time = (kinematic_data[joint] >= max_threshold).mean() * 100
        extreme_min_time = (kinematic_data[joint] <= min_threshold).mean() * 100
        
        # Sum of times at both high and low extremes
        total_extreme_time = extreme_max_time + extreme_min_time
        
        results[f'{joint}_extremity_index'] = [total_extreme_time]
    
    # Composite extremity index as the average of individual indices
    results['composite_extremity_index'] = results.mean(axis=1)
    
    return results



def rom_progression_metric(kinematic_data, joint_cols, window_size=60):
    """
    Track the progression of maximum joint angles over time.
    
    Parameters:
    - kinematic_data: DataFrame containing joint angle data.
    - joint_cols: List of column names for joint angles.
    - window_size: Size of the rolling window (in samples or seconds).
    
    Returns:
    - DataFrame with maximum joint angles and progression metrics.
    """
    results = pd.DataFrame()
    
    for joint in joint_cols:
        max_angles = []
        # Compute the maximum angle in each window segment
        for i in range(window_size, len(kinematic_data), window_size//2):
            segment = kinematic_data[joint].iloc[max(0, i-window_size):i]
            if len(segment) >= window_size//2:
                max_angle = segment.max()
                max_angles.append(max_angle)
            else:
                max_angles.append(np.nan)
                
        results[f'{joint}_max_angle'] = max_angles
        
        # Calculate progression: relative change from the initial maximum angle
        if len(max_angles) > 1 and max_angles[0] != 0:
            results[f'{joint}_rom_progression'] = [angle / max_angles[0] - 1 for angle in max_angles]
        else:
            results[f'{joint}_rom_progression'] = [0] * len(max_angles)
    
    # Composite metric: average progression across the joints
    results['composite_rom_progression'] = results[[f'{joint}_rom_progression' for joint in joint_cols]].mean(axis=1)
    
    return results



def calculate_muscle_length(kinematic_data, joints, muscle):
    """
    Placeholder function to calculate muscle length based on joint angles.
    In a real application, this function should implement a biomechanical model.
    
    Parameters:
    - kinematic_data: DataFrame containing joint angle data.
    - joints: List of joint column names that influence the muscle's length.
    - muscle: Name of the muscle (for potential specific adjustments).
    
    Returns:
    - Series representing the calculated muscle length over time.
    """
    # Simple example: sum of joint angles (not a true biomechanical measure)
    return kinematic_data[joints].sum(axis=1)

def muscle_overextension_index(kinematic_data, joint_muscle_map, rest_lengths, critical_threshold=1.2):
    """
    Calculate the percentage of time muscles spend beyond a critical length threshold.
    
    Parameters:
    - kinematic_data: DataFrame containing joint angle data.
    - joint_muscle_map: Dictionary mapping each muscle to the list of joint columns affecting its length.
    - rest_lengths: Dictionary of rest lengths for each muscle.
    - critical_threshold: Proportion of the rest length defining over-extension (e.g., 1.2 for 120%).
    
    Returns:
    - DataFrame with overextension indices and maximum length ratios for each muscle, along with a composite index.
    """
    results = pd.DataFrame()
    muscle_lengths = {}
    
    # Calculate estimated muscle lengths using the provided biomechanical (or placeholder) model
    for muscle, joints in joint_muscle_map.items():
        muscle_lengths[muscle] = calculate_muscle_length(kinematic_data, joints, muscle)
    
    # Compute over-extension metrics for each muscle
    for muscle, length_series in muscle_lengths.items():
        critical_length = rest_lengths[muscle] * critical_threshold
        overextension_time = (length_series >= critical_length).mean() * 100
        results[f'{muscle}_overextension_index'] = [overextension_time]
        
        # Maximum ratio of muscle length to its rest length
        max_length_ratio = length_series.max() / rest_lengths[muscle]
        results[f'{muscle}_max_length_ratio'] = [max_length_ratio]
    
    # Composite index: average overextension across all muscles
    results['composite_overextension_index'] = results[[f'{muscle}_overextension_index' for muscle in joint_muscle_map.keys()]].mean(axis=1)
    
    return results


def velocity_at_extreme_rom(kinematic_data, joint_cols, velocity_cols, percentile_threshold=90):
    """
    Calculate angular velocities when joints are at extreme ranges of motion.
    
    Parameters:
    - kinematic_data: DataFrame containing joint angle and velocity data.
    - joint_cols: List of joint angle column names.
    - velocity_cols: List of corresponding angular velocity column names.
    - percentile_threshold: Percentile used to define extreme ROM.
    
    Returns:
    - DataFrame with mean and maximum velocities at extreme ROM for each joint,
      as well as a composite metric.
    """
    results = pd.DataFrame()
    
    for joint, velocity in zip(joint_cols, velocity_cols):
        # Define extreme ROM threshold for the joint
        extreme_threshold = np.percentile(kinematic_data[joint], percentile_threshold)
        
        # Identify data points where joint is at or beyond the extreme threshold
        extreme_rom_mask = kinematic_data[joint] >= extreme_threshold
        
        if extreme_rom_mask.sum() > 0:
            mean_velocity = kinematic_data.loc[extreme_rom_mask, velocity].abs().mean()
            max_velocity = kinematic_data.loc[extreme_rom_mask, velocity].abs().max()
        else:
            mean_velocity = 0
            max_velocity = 0
            
        results[f'{joint}_mean_velocity_at_extreme'] = [mean_velocity]
        results[f'{joint}_max_velocity_at_extreme'] = [max_velocity]
    
    # Composite metric: average mean velocity at extreme ROM across joints
    results['composite_velocity_at_extreme'] = results[[f'{joint}_mean_velocity_at_extreme' for joint in joint_cols]].mean(axis=1)
    
    return results




if __name__ == "__main__":
    # First, run the detailed EDA on your dataset.
    # detailed_eda(granular_data, dataset_name="Granular Dataset")
    
    # ---------------------
    # 1. Compute EMG-Based Fatigue Features
    # ---------------------
    # Define the EMG columns for the three muscles (FDS, FCU, FCR)
    emg_cols_FDS = "EMG 1 (mV) - FDS (81770)"
    emg_cols_FCU = "EMG 1 (mV) - FCU (81728)"
    emg_cols_FCR = "EMG 1 (mV) - FCR (81745)"
    muscle_cols = [emg_cols_FDS, emg_cols_FCU, emg_cols_FCR]
    
    # Use a rolling window of 60 samples (or seconds, if your sampling is in seconds)
    decay_data = activation_decay_index(granular_data, muscle_cols=muscle_cols, window_size=60)
    freq_data = frequency_shift_index(granular_data, muscle_cols=muscle_cols, window_size=60, sampling_rate=1000)
    
    # For force–activation efficiency, we assume there is a force measurement.
    # If not available, we use a dummy force vector (all ones).
    if "Force (N)" in granular_data.columns:
        force_data = granular_data["Force (N)"]
    else:
        force_data = pd.Series(np.ones(len(granular_data)), index=granular_data.index)
    efficiency_data = force_activation_efficiency(granular_data, force_data, muscle_cols=muscle_cols, window_size=60)
    
    # Combine the above three features into a composite exhaustion index.
    exhaustion_index = composite_exhaustion_index(decay_data, freq_data, efficiency_data)
    
    # ---------------------
    # 2. Compute Kinematic Features (for ongoing time series metrics)
    # ---------------------
    # Assume that the elbow joint angles are used for tracking range of motion.
    joint_cols = ["elbow_angle_x_biomech", "elbow_angle_y_biomech", "elbow_angle_z_biomech"]
    extremity_index = joint_angle_extremity_index(granular_data, joint_cols=joint_cols, percentile_threshold=90)
    rom_progression = rom_progression_metric(granular_data, joint_cols=joint_cols, window_size=60)
    
    # For muscle length overextension, we provide a dummy joint-muscle map and rest-lengths.
    joint_muscle_map = {
        "FDS": ["elbow_angle_x_biomech", "elbow_angle_y_biomech"],
        "FCU": ["elbow_angle_x_biomech", "elbow_angle_y_biomech"],
        "FCR": ["elbow_angle_x_biomech", "elbow_angle_y_biomech"]
    }
    rest_lengths = {"FDS": 1.0, "FCU": 1.0, "FCR": 1.0}  # Dummy rest lengths; update as needed.
    overextension_data = muscle_overextension_index(granular_data, joint_muscle_map, rest_lengths, critical_threshold=1.2)
    
    # Compute joint velocity at extreme ROM for the elbow.
    # We assume that "elbow_angle_z_biomech" is our joint measure and "elbow_velo_z_biomech" its velocity.
    elbow_joint_col = "elbow_angle_z_biomech"
    elbow_velocity_col = "elbow_velo_z_biomech"
    if elbow_velocity_col in granular_data.columns:
        velocity_data = velocity_at_extreme_rom(granular_data, 
                                                joint_cols=[elbow_joint_col], 
                                                velocity_cols=[elbow_velocity_col], 
                                                percentile_threshold=90)
    else:
        velocity_data = pd.DataFrame({f'{elbow_joint_col}_mean_velocity_at_extreme': [np.nan]})
    
    # ---------------------
    # 3. Define Injury Indicators
    # ---------------------
    # (a) Elbow Injury Indicator:
    # Flag an injury risk if the elbow angle (using z-axis as an example) exceeds a threshold (e.g., 120°).
    def injury_indicator_elbow(kinematic_data, elbow_angle_col="elbow_angle_z_biomech", threshold=120):
        """
        Returns a binary flag (1 = injury risk) if the elbow angle exceeds the threshold.
        """
        return (kinematic_data[elbow_angle_col] > threshold).astype(int)
    
    granular_data["elbow_injury_flag"] = injury_indicator_elbow(granular_data, 
                                                                 elbow_angle_col="elbow_angle_z_biomech", 
                                                                 threshold=120)
    
    # (b) Muscle Injury Indicator:
    # Flag an injury risk if the composite exhaustion index exceeds a given threshold.
    # (Threshold value may require tuning based on your data/distribution.)
    def injury_indicator_muscle(exhaustion_index, threshold=1.0):
        """
        Returns a binary flag (1 = injury risk) if the composite exhaustion index exceeds the threshold.
        """
        return (exhaustion_index > threshold).astype(int)
    
    granular_data["muscle_injury_flag"] = injury_indicator_muscle(exhaustion_index, threshold=1.0)
    
    # ---------------------
    # 4. Output and Integration
    # ---------------------
    # For demonstration purposes, we print the head of each computed metric.
    print("\nComposite Exhaustion Index (first few values):")
    print(exhaustion_index.head())
    
    print("\nJoint Angle Extremity Index (first few values):")
    print(extremity_index.head())
    
    print("\nROM Progression Metric (first few values):")
    print(rom_progression.head())
    
    print("\nMuscle Overextension Data (first few values):")
    print(overextension_data.head())
    
    print("\nVelocity at Extreme ROM for Elbow (first few values):")
    print(velocity_data.head())
    
    print("\nElbow Injury Flag (distribution):")
    print(granular_data["elbow_injury_flag"].value_counts())
    
    print("\nMuscle Injury Flag (distribution):")
    print(granular_data["muscle_injury_flag"].value_counts())
    
    # (Optional) Save the updated dataset with new metrics/indicators for further machine learning work.
    # granular_data.to_csv("granular_data_with_features_and_injury_flags.csv", index=False)
    
    # Now, granular_data along with the computed features (which you can also align based on time)
    # can be used for time series machine learning.





Composite Exhaustion Index (first few values):
0         NaN
1         NaN
2   -0.314464
3   -1.866018
4   -0.239205
dtype: float64

Joint Angle Extremity Index (first few values):
   elbow_angle_x_biomech_extremity_index  \
0                                   20.0   

   elbow_angle_y_biomech_extremity_index  \
0                                   20.0   

   elbow_angle_z_biomech_extremity_index  composite_extremity_index  
0                                   20.0                       20.0  

ROM Progression Metric (first few values):
   elbow_angle_x_biomech_max_angle  elbow_angle_x_biomech_rom_progression  \
0                        42.861435                               0.000000   
1                        43.087118                               0.005265   
2                        43.312801                               0.010531   
3                        43.539745                               0.015826   
4                        43.765428                               0.0210